In [2]:
import tensorflow as tf

import numpy as np
from tensorflow.keras import layers
import tensorflow.keras.backend as K
import functions.getLayerOutput as glo

import ssl
from tensorflow.keras import datasets, layers, models

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [4]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [5]:
NUM_EPOCH = 200

# training SNR
eb_n0 = 10
channel_snr = 10
rate = 1 # code rate 
channel_type = 'fading'


#trn, tst = utils.get_mnist()

# ##### KSG MI estimator #######################
# settings = {'kraskov_k': 3}
# gpu_est = est.OpenCLKraskovMI(settings = settings)
# # usage: MI = estimator.estimate(var1, var2)

##### setting for communication channel ######

def getnoisevariance(SNR,rate,P=1):
    # the SNR in args[0] is actually EbN0
    snrdB = SNR + 10*np.log10(rate)
    snr = 10.0**(snrdB/10.0)
    #P_avg = 1
    N0 = P/snr
    return (N0/2)

noise_var = getnoisevariance(eb_n0,rate,P=1)

##############################################


##############################################

label_permutation = np.arange(10)
LABEL_FIRST_HALF = label_permutation[:5]
label2_list = np.zeros((6,5))
for i in range(6):
    label2_list[i][:] = label_permutation[i:i+5]
#LABEL_SECOND_HALF = [5,6,7,8,9]

In [6]:

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

x_train = (x_train / 255.0)
x_test = (x_test / 255.0)
# reshape to 1d
x_train = x_train.reshape(x_train.shape[0], -1)
x_test = x_test.reshape(x_test.shape[0], -1)

# save: original classification
Y_train = tf.keras.utils.to_categorical(y_train, 10)
Y_test = tf.keras.utils.to_categorical(y_test, 10)


4422102/4422102 [==============================] - 2s 0us/step


In [6]:
x_train.shape

(60000, 784)

In [7]:
train_filter = np.where(np.in1d(y_train, LABEL_FIRST_HALF))
test_filter =  np.where(np.in1d(y_test, LABEL_FIRST_HALF))

x_1train = x_train[train_filter]
Y_1train = Y_train[train_filter]
x_1test = x_test[test_filter]
Y_1test = Y_test[test_filter]

In [8]:
### normalized Input before 
# class GaussianNoiseLayer(layers.Layer):
#     def __init__(self, stddev, **kwargs):
#         super(GaussianNoiseLayer, self).__init__(**kwargs)
#         self.stddev = stddev
    
#     def call(self, inputs, training=None):
#         if training:
#             noise = tf.random.normal(shape=tf.shape(inputs), mean=0.0, stddev=self.stddev)
#             return inputs + noise
#         else:
#             noise = tf.random.normal(shape=tf.shape(inputs), mean=0.0, stddev=self.stddev)
#             return inputs + noise
    
#     def get_config(self):
#         config = super(GaussianNoiseLayer, self).get_config()
#         config.update({'noise_var': self.stddev})
#         return config

In [8]:
## fading channel

def fading(x, stddev, h=None):
    """Implements the fading channel with multiplicative fading and
    additive white gaussian noise.
    Args:
        x: channel input symbols
        stddev: standard deviation of noise
    Returns:
        y: noisy channel output symbols
    """
    # channel gain
    if h is None:
        h = tf.complex(
            tf.random.normal([tf.shape(x)[0], 1], 0, 1 / np.sqrt(2)),
            tf.random.normal([tf.shape(x)[0], 1], 0, 1 / np.sqrt(2)),
        )

    # additive white gaussian noise
    awgn = tf.complex(
        tf.random.normal(tf.shape(x), 0, 1 / np.sqrt(2)),
        tf.random.normal(tf.shape(x), 0, 1 / np.sqrt(2)),
    )

    return (h * x + stddev * awgn), h

class Channel(layers.Layer):

#cite D-JSCC

    def __init__(self, channel_type, channel_snr, name="channel", **kwargs):
        super(Channel, self).__init__(name=name, **kwargs)
        self.channel_type = channel_type
        self.channel_snr = channel_snr

    def call(self, inputs):
        (encoded_img, prev_h) = inputs
        inter_shape = tf.shape(encoded_img)
        # reshape array to [-1, dim_z]
        z = layers.Flatten()(encoded_img)
        # convert from snr to std
        # print("channel_snr: {}".format(self.channel_snr))
        noise_stddev = np.sqrt(10 ** (-self.channel_snr / 10))

        # Add channel noise
        if self.channel_type == "awgn":
            dim_z = tf.shape(z)[1]
            # normalize latent vector so that the average power is 1
            z_in = tf.sqrt(tf.cast(dim_z, dtype=tf.float32)) * tf.nn.l2_normalize(
                z, axis=1
            )
            z_out = real_awgn(z_in, noise_stddev)
            h = tf.ones_like(z_in)  # h just makes sense on fading channels

        elif self.channel_type == "fading":
            dim_z = tf.shape(z)[1] // 2
            # convert z to complex representation
            z_in = tf.complex(z[:, :dim_z], z[:, dim_z:])
            # normalize the latent vector so that the average power is 1
            z_norm = tf.reduce_sum(
                tf.math.real(z_in * tf.math.conj(z_in)), axis=1, keepdims=True
            )
            z_in = z_in * tf.complex(
                tf.sqrt(tf.cast(dim_z, dtype=tf.float32) / z_norm), 0.0
            )
            z_out, h = fading(z_in, noise_stddev, prev_h)
            # convert back to real
            z_out = tf.concat([tf.math.real(z_out), tf.math.imag(z_out)], 1)

        # convert signal back to intermediate shape
        z_out = tf.reshape(z_out, inter_shape)

        return z_out, h


In [9]:
class CustomMetricCallback(tf.keras.callbacks.Callback):
    def __init__(self, display_frequency=10):
        super(CustomMetricCallback, self).__init__()
        self.display_frequency = display_frequency

    def on_epoch_end(self, epoch, logs=None):
        if epoch % self.display_frequency == 0:
            metrics_str = " - ".join([f"{metric_name}: {value:.4f}" for metric_name, value in logs.items()])
            print(f"Epoch {epoch}/{self.params['epochs']} - {metrics_str}")

            
callback = CustomMetricCallback(display_frequency=50)

In [11]:
SAMPLE_NUM = 1000
EPOCHS = 2


def do_report(epoch):
    if epoch == (EPOCHS-1):
        return True

    
    
def train_first_stage(lambda_val):

    tf.keras.backend.clear_session()
    tf.random.set_seed(42)
    prev_chn_gain = None

        ############ Model Structure ###############################

    input_layer  = tf.keras.layers.Input((x_train.shape[1],))
    encoder_1 = tf.keras.layers.Dense(128, activation='relu')(input_layer)
    encoder_2 =  tf.keras.layers.Dense(80, activation='relu')(encoder_1)
    ## first loss: classification 
    normalized_x = tf.keras.layers.Lambda(lambda x: K.tanh(x))(encoder_2)
    fadingchannel = Channel(channel_type, channel_snr, name="channel_output")
    noise_layer, chn_gain = fadingchannel((normalized_x, prev_chn_gain))

    CE_decoder_1 = tf.keras.layers.Dense(80, activation='relu')(noise_layer)
    CE_decoder_2 = tf.keras.layers.Dense(80, activation='relu')(CE_decoder_1)
    CE_output = tf.keras.layers.Dense(10,activation = 'softmax',name='CE')(CE_decoder_2)

    ## second loss: reconstruction
    mse_decoder_1 = tf.keras.layers.Dense(256, activation='relu')(noise_layer)
    mse_output = tf.keras.layers.Dense(x_train.shape[1],activation = 'sigmoid',name='mse')(mse_decoder_1) 
    
    #    mse_decoder_1 = tf.keras.layers.Dense(256, activation='relu')(noise_layer)
    #    mse_output = tf.keras.layers.Dense(x_train.shape[1],activation = 'sigmoid',name='mse')(mse_decoder_1)  

    model = tf.keras.Model(inputs = input_layer, outputs = [CE_output,mse_output])
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=5e-2,
    decay_steps=10000,
    decay_rate=0.8)
    opt = tf.keras.optimizers.SGD(learning_rate=lr_schedule)
    model.compile(optimizer=opt,
                  loss = {'CE' : 'categorical_crossentropy', 
                          'mse' : 'mse'},
                  metrics = {'CE' : 'accuracy', 
                             'mse': tf.keras.metrics.RootMeanSquaredError()},
                  loss_weights=[1, lambda_val])


    SNR = eb_n0
    #     print("====================")
    reporter = glo.getMIOutput(trn=[x_1train[:SAMPLE_NUM]], 
               num_selection = SAMPLE_NUM,
               embedding_network = model,
               target_layer = 2,
               do_save_func=do_report)

    history = model.fit(x=x_1train, y=(Y_1train,x_1train),
                        batch_size=128,
                        epochs=2,
                        verbose=1,
                        validation_data=(x_1test, (Y_1test,x_1test)),
                        callbacks=[reporter,])
    print("acc:", history.history['val_CE_accuracy'][-1]) 
    
    Z = reporter.datalist[-1]['data']['activity_tst'][-1] ## change 99 to -1
    current_iteration = {
    "data": Z
        }


#     ep_last = len(reporter.datalist) -1
#     # Z_data = reporter.datalist[EPOCH]['data']['activity_tst'][3][:10000]
#     ZHAT_data = reporter.datalist[ep_last]['data']['activity_tst'][4][:SAMPLE_NUM]
#     x_selected = funcs.normalize_data_new(x_1train[:SAMPLE_NUM],C=10)[0]
#     print("Current I(Z_hat; X) is: ",gpu_est.estimate(funcs.normalize_data_new(ZHAT_data,C=4)[0],x_selected))

  
    ################### secpmd stage #################
    print("stage 2!")
   
    train_filter = np.where(np.in1d(y_train, LABEL_SECOND_HALF))
    test_filter =  np.where(np.in1d(y_test, LABEL_SECOND_HALF))

    x_retrain = x_train[train_filter]
    x_retest = x_test[test_filter]
    Y_retrain = Y_train[train_filter]
    Y_retest = Y_test[test_filter]

    CE_dense_3 = layers.Dense(40, activation='relu')(noise_layer)
    CE_dense_3 = layers.Dense(40, activation='relu')(CE_dense_3)
    CE_output1 = tf.keras.layers.Dense(10,activation = 'softmax',name='CE')(CE_dense_3)
    #     CE_decoder_3 = tf.keras.layers.Dense(64, activation='relu')(noise_layer)
    #     CE_decoder_4 = tf.keras.layers.Dense(64, activation='relu')(CE_decoder_3)
    #     CE_output1 = tf.keras.layers.Dense(10, activation='softmax', name='CE')(CE_decoder_4)

    reconstructed_model = tf.keras.models.Model(inputs = input_layer, outputs = [CE_output1,mse_output])

    reconstructed_model.layers[1].trainable = False
    reconstructed_model.layers[2].trainable = False

    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=(5e-2)*2,
    decay_steps=10000,
    decay_rate=0.8)
    opt = tf.keras.optimizers.SGD(learning_rate=lr_schedule)


    reconstructed_model.compile(optimizer=opt, 
                loss = {'CE' : 'categorical_crossentropy', 
                          'mse' : 'mse'
                         },              loss_weights=[1, 0],
                  metrics = {'CE' : 'accuracy', 
                             'mse': tf.keras.metrics.RootMeanSquaredError()
                           }
                 )

    #     reporter_2 = getMIOutput(trn=x_retrain, 
    #                                tst=Y_retrain, 
    #                                snr=SNR,
    #                               do_save_func=do_report)

    history1 = reconstructed_model.fit(x=x_retrain, y=(Y_retrain,x_retrain),
                        batch_size=256,
                        epochs=100,
                        verbose=0,
                        validation_data=(x_retest, (Y_retest,x_retest)))
                        # callbacks=[callback])
    print("acc:", history1.history['val_CE_accuracy'][-1]) 
    return history.history['val_CE_accuracy'][-1], history1.history['val_CE_accuracy'][-1]
    

In [18]:
Z

NameError: name 'Z' is not defined

In [17]:
sd = np.sqrt(noise_var)

lambda_list = [0,1,3,10]
acc1_list = []
acc2_list = []
for i in range(6):
    print("current overlap is: ", (i) )
    LABEL_SECOND_HALF = label2_list[5-i]
    for lambda_val in lambda_list:
        print("lambda: ",lambda_val)
        acc1, acc2 = train_first_stage(lambda_val)
        acc1_list.append(acc1)
        acc2_list.append(acc2)
        #train_second_stage()

current overlap is:  0
lambda:  0
Epoch 1/2
240/240 [==============================] - 2s 5ms/step - loss: 1.0660 - CE_loss: 1.0660 - mse_loss: 0.2331 - CE_accuracy: 0.7078 - mse_root_mean_squared_error: 0.4828 - val_loss: 0.4691 - val_CE_loss: 0.4691 - val_mse_loss: 0.2332 - val_CE_accuracy: 0.9066 - val_mse_root_mean_squared_error: 0.4830
Epoch 2/2
240/240 [==============================] - 1s 4ms/step - loss: 0.3523 - CE_loss: 0.3523 - mse_loss: 0.2331 - CE_accuracy: 0.9232 - mse_root_mean_squared_error: 0.4828 - val_loss: 0.2688 - val_CE_loss: 0.2688 - val_mse_loss: 0.2333 - val_CE_accuracy: 0.9401 - val_mse_root_mean_squared_error: 0.4830
acc: 0.9400661587715149
stage 2!


Exception ignored in: <function WeakKeyDictionary.__init__.<locals>.remove at 0x7fe349feecb0>
Traceback (most recent call last):
  File "/usr/lib/python3.10/weakref.py", line 370, in remove
    def remove(k, selfref=ref(self)):
KeyboardInterrupt: 


KeyboardInterrupt: 

## MI estiamtion

In [10]:
SAMPLE_NUM = 10000
EPOCHS = 100


def do_report(epoch):
    if epoch == (EPOCHS-1):
        return True

    
    
def train_first_stage(lambda_val):

    tf.keras.backend.clear_session()
    tf.random.set_seed(42)
    prev_chn_gain = None

        ############ Model Structure ###############################

    input_layer  = tf.keras.layers.Input((x_train.shape[1],))
    encoder_1 = tf.keras.layers.Dense(128, activation='relu')(input_layer)
    encoder_2 =  tf.keras.layers.Dense(80, activation='relu')(encoder_1)
    ## first loss: classification 
    normalized_x = tf.keras.layers.Lambda(lambda x: K.tanh(x))(encoder_2)
    fadingchannel = Channel(channel_type, channel_snr, name="channel_output")
    noise_layer, chn_gain = fadingchannel((normalized_x, prev_chn_gain))

    CE_decoder_1 = tf.keras.layers.Dense(80, activation='relu')(noise_layer)
    CE_decoder_2 = tf.keras.layers.Dense(80, activation='relu')(CE_decoder_1)
    CE_output = tf.keras.layers.Dense(10,activation = 'softmax',name='CE')(CE_decoder_2)

    ## second loss: reconstruction
    mse_decoder_1 = tf.keras.layers.Dense(256, activation='relu')(noise_layer)
    mse_output = tf.keras.layers.Dense(x_train.shape[1],activation = 'sigmoid',name='mse')(mse_decoder_1) 
    
    #    mse_decoder_1 = tf.keras.layers.Dense(256, activation='relu')(noise_layer)
    #    mse_output = tf.keras.layers.Dense(x_train.shape[1],activation = 'sigmoid',name='mse')(mse_decoder_1)  

    model = tf.keras.Model(inputs = input_layer, outputs = [CE_output,mse_output])
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=5e-2,
    decay_steps=10000,
    decay_rate=0.8)
    opt = tf.keras.optimizers.SGD(learning_rate=lr_schedule)
    model.compile(optimizer=opt,
                  loss = {'CE' : 'categorical_crossentropy', 
                          'mse' : 'mse'},
                  metrics = {'CE' : 'accuracy', 
                             'mse': tf.keras.metrics.RootMeanSquaredError()},
                  loss_weights=[1, lambda_val])


    SNR = eb_n0
    #     print("====================")
    reporter = glo.getMIOutput(trn=[x_1train[:SAMPLE_NUM]], 
               num_selection = SAMPLE_NUM,
               embedding_network = model,
               target_layer = 2,
               do_save_func=do_report)

    history = model.fit(x=x_1train, y=(Y_1train,x_1train),
                        batch_size=128,
                        epochs=EPOCHS,
                        verbose=0,
                        validation_data=(x_1test, (Y_1test,x_1test)),
                        callbacks=[reporter,])
    print("acc:", history.history['val_CE_accuracy'][-1]) 
    
    Z = reporter.datalist[-1]['data']['activity_tst'][-1] ## change 99 to -1
#     current_iteration = {
#     "data": Z
#         }
    return Z

#     ep_last = len(reporter.datalist) -1
#     # Z_data = reporter.datalist[EPOCH]['data']['activity_tst'][3][:10000]
#     ZHAT_data = reporter.datalist[ep_last]['data']['activity_tst'][4][:SAMPLE_NUM]
#     x_selected = funcs.normalize_data_new(x_1train[:SAMPLE_NUM],C=10)[0]
#     print("Current I(Z_hat; X) is: ",gpu_est.estimate(funcs.normalize_data_new(ZHAT_data,C=4)[0],x_selected))

  
#     ################### secpmd stage #################
#     print("stage 2!")
   
#     train_filter = np.where(np.in1d(y_train, LABEL_SECOND_HALF))
#     test_filter =  np.where(np.in1d(y_test, LABEL_SECOND_HALF))

#     x_retrain = x_train[train_filter]
#     x_retest = x_test[test_filter]
#     Y_retrain = Y_train[train_filter]
#     Y_retest = Y_test[test_filter]

#     CE_dense_3 = layers.Dense(40, activation='relu')(noise_layer)
#     CE_dense_3 = layers.Dense(40, activation='relu')(CE_dense_3)
#     CE_output1 = tf.keras.layers.Dense(10,activation = 'softmax',name='CE')(CE_dense_3)
#     #     CE_decoder_3 = tf.keras.layers.Dense(64, activation='relu')(noise_layer)
#     #     CE_decoder_4 = tf.keras.layers.Dense(64, activation='relu')(CE_decoder_3)
#     #     CE_output1 = tf.keras.layers.Dense(10, activation='softmax', name='CE')(CE_decoder_4)

#     reconstructed_model = tf.keras.models.Model(inputs = input_layer, outputs = [CE_output1,mse_output])

#     reconstructed_model.layers[1].trainable = False
#     reconstructed_model.layers[2].trainable = False

#     lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate=(5e-2)*2,
#     decay_steps=10000,
#     decay_rate=0.8)
#     opt = tf.keras.optimizers.SGD(learning_rate=lr_schedule)


#     reconstructed_model.compile(optimizer=opt, 
#                 loss = {'CE' : 'categorical_crossentropy', 
#                           'mse' : 'mse'
#                          },              loss_weights=[1, 0],
#                   metrics = {'CE' : 'accuracy', 
#                              'mse': tf.keras.metrics.RootMeanSquaredError()
#                            }
#                  )

#     #     reporter_2 = getMIOutput(trn=x_retrain, 
#     #                                tst=Y_retrain, 
#     #                                snr=SNR,
#     #                               do_save_func=do_report)

#     history1 = reconstructed_model.fit(x=x_retrain, y=(Y_retrain,x_retrain),
#                         batch_size=256,
#                         epochs=100,
#                         verbose=0,
#                         validation_data=(x_retest, (Y_retest,x_retest)))
#                         # callbacks=[callback])
#     print("acc:", history1.history['val_CE_accuracy'][-1]) 
#     return history.history['val_CE_accuracy'][-1], history1.history['val_CE_accuracy'][-1]
    

In [11]:
sd = np.sqrt(noise_var)

lambda_list = [0,1,3,10]
z_list = []

for i in range(6):
    print("current overlap is: ", (i) )
    LABEL_SECOND_HALF = label2_list[5-i]
    for lambda_val in lambda_list:
        print("lambda: ",lambda_val)
        z = train_first_stage(lambda_val)
        z_list.append(z)

        #train_second_stage()

current overlap is:  0
lambda:  0


2024-07-30 07:33:02.062253: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1638] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78835 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:0f:00.0, compute capability: 8.0
2024-07-30 07:33:04.753966: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:655] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-07-30 07:33:04.807757: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x5586db57f380 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-07-30 07:33:04.807797: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA A100-SXM4-80GB, Compute Capability 8.0
2024-07-30 07:33:04.828527: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8902
2024-07-30 07:33:04.896253: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled clus

acc: 0.9025999903678894
lambda:  1
acc: 0.8877999782562256
lambda:  3
acc: 0.8889999985694885
lambda:  10
acc: 0.8971999883651733
current overlap is:  1
lambda:  0
acc: 0.9028000235557556
lambda:  1
acc: 0.8934000134468079
lambda:  3
acc: 0.8873999714851379
lambda:  10
acc: 0.8978000283241272
current overlap is:  2
lambda:  0
acc: 0.8949999809265137
lambda:  1
acc: 0.8949999809265137
lambda:  3
acc: 0.8885999917984009
lambda:  10
acc: 0.897599995136261
current overlap is:  3
lambda:  0
acc: 0.8980000019073486
lambda:  1
acc: 0.8974000215530396
lambda:  3
acc: 0.8848000168800354
lambda:  10
acc: 0.8863999843597412
current overlap is:  4
lambda:  0
acc: 0.8827999830245972
lambda:  1
acc: 0.8953999876976013
lambda:  3
acc: 0.8925999999046326
lambda:  10
acc: 0.8889999985694885
current overlap is:  5
lambda:  0
acc: 0.9034000039100647
lambda:  1
acc: 0.8899999856948853
lambda:  3
acc: 0.8880000114440918
lambda:  10
acc: 0.8916000127792358


In [15]:
from functions.MIestimate import MINE_MI
from functions.MIestimate import global_normalize_mine as VN


In [16]:
x_selected = VN(x_1train[:SAMPLE_NUM],C=1)

In [17]:

##### TEST THE RANGE FOR PHASE 2

izx_val_C_z=[]
se = []

count = 0
#dnew=z_selected
for i in range(6):
    print("current overlap is: ", (i) )
    for lambda_val in lambda_list:
        print("lambda: ", lambda_val)
        z = z_list[count]
        dnew = VN(z,C=1)

        #dnew = z[:SAMPLE_NUM]
        normz = VN(dnew,C=1)
        izx_val,_ = MINE_MI(dnew,x_selected,total_epochs=50)

        print("I_Z_X: ",izx_val)
        #print("I_Z_Y: ",izy_val)
        izx_val_C_z.append(izx_val)
        count+=1


current overlap is:  0
lambda:  0


/usr/local/lib/python3.10/dist-packages/keras/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(
2024-07-30 08:48:23.220034: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.


I_Z_X:  3.4271302725162482
lambda:  1


I_Z_X:  3.519193273453713
lambda:  3
I_Z_X:  3.6968056044098647
lambda:  10


I_Z_X:  4.349830265076682
current overlap is:  1
lambda:  0


...

I_Z_X:  4.015169912693346
lambda:  10


I_Z_X:  4.762281810098283
current overlap is:  5
lambda:  0


I_Z_X:  3.6173472119541352
lambda:  1


I_Z_X:  3.9502935852389722
lambda:  3


I_Z_X:  3.760189810385353
lambda:  10
I_Z_X:  4.648511846899519


In [19]:
np.mean(np.reshape(izx_val_C_z,(6,-1)),axis=0)

array([3.88520205, 3.78225943, 3.8621804 , 4.64042763])

In [22]:
np.reshape(izx_val_C_z,(6,-1))

array([[3.42713027, 3.51919327, 3.6968056 , 4.34983027],
       [3.98867495, 3.81593352, 3.86476283, 4.71510147],
       [4.20078597, 3.42101285, 3.90127268, 4.65850232],
       [4.11385321, 3.8597485 , 3.93488153, 4.70833808],
       [3.96342068, 4.12737483, 4.01516991, 4.76228181],
       [3.61734721, 3.95029359, 3.76018981, 4.64851185]])

In [1]:
array([2.8692471 , 2.3776179 , 2.40168653, 3.04009828])

NameError: name 'np' is not defined

In [75]:
# Save the entire list using pickle
import pickle
with open('z_mnist_fading.pkl', 'wb') as f:
    pickle.dump(z_list, f)

In [65]:
def global_normalize_mine(data_orig, C):
    # Reshape data as before
    data = np.reshape(data_orig, (data_orig.shape[0], int(data_orig.size / data_orig.shape[0])))
    
    # Subtract the mean
    means = np.mean(data, axis=0)
    data = np.abs(data - means)
    
    # Compute the normalization factor
    # norm = np.sqrt(np.mean(np.sum(data ** 2, axis=1)))
    # linalg.norm(x, ord=None, 
    norm = np.sqrt(np.mean(np.sum(data ** 2, axis=1)))
    norm = norm/ np.sqrt(data.shape[1])
    
    # print(norm)
    # Normalize the data
    
    data = C*data_orig / norm
    return data

In [74]:

##### TEST THE RANGE FOR PHASE 2

izx_val_C_z=[]
se = []

count = 0
#dnew=z_selected
for i in range(6):
    print("current overlap is: ", (i) )
    for lambda_val in lambda_list:
        print("lambda: ", lambda_val)
        z = z_list[count]
        dnew = VN(z,C=1)[0]

        #dnew = z[:SAMPLE_NUM]
        
        #normz,_ = VN(dnew,C=1)
        izx_val,_ = MINE_MI(dnew,x_selected,total_epochs=50)

        print("I_Z_X: ",izx_val)
        #print("I_Z_Y: ",izy_val)
        izx_val_C_z.append(izx_val)
        count+=1



current overlap is:  0
lambda:  0


I_Z_X:  1.148905161604618
lambda:  1


I_Z_X:  0.6850456374786736
lambda:  3


I_Z_X:  1.5967646667685054
lambda:  10


I_Z_X:  2.3628347277260024
current overlap is:  1
lambda:  0


...

I_Z_X:  1.4207324264171555
lambda:  10


I_Z_X:  2.4810889665633646
current overlap is:  5
lambda:  0


I_Z_X:  1.329271278766385
lambda:  1


I_Z_X:  0.5832660467240872
lambda:  3


I_Z_X:  1.6243355494936238
lambda:  10
I_Z_X:  2.482817048239234


In [13]:
np.reshape(izx_val_C_z,(6,-1))

NameError: name 'izx_val_C_z' is not defined

In [76]:
np.reshape(izx_val_C_z,(6,-1))

array([[1.14890516, 0.68504564, 1.59676467, 2.36283473],
       [1.11080094, 0.6105913 , 1.57846313, 2.57590747],
       [1.1022909 , 0.78434554, 1.51329719, 2.58763703],
       [0.86338765, 0.33282115, 1.5714655 , 2.18869473],
       [1.34816022, 0.3386051 , 1.42073243, 2.48108897],
       [1.32927128, 0.58326605, 1.62433555, 2.48281705]])

In [77]:
np.mean(np.reshape(izx_val_C_z,(6,-1)),axis=0)

array([1.15046936, 0.55577913, 1.55084308, 2.44649666])